<a href="https://colab.research.google.com/github/mohamedSalehMatar/EnhancedTemplateMatcher/blob/main/EnhancedTemplateMatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import cv2
import numpy as np
from scipy import ndimage
from google.colab.patches import cv2_imshow
from google.colab import files

In [10]:
# The function works the same as range() function in Python but with float numbers
def frange(start, end, step):
    current = start
    while current < end:
        yield current
        current += step

In [11]:
# Difining the global variable needed
best_match = None
best_val = -1
best_scale = 0.0
best_angle = 0.0

In [ ]:
# Upload the template image and gray-scaling it

print("Upload the template image:")
uploaded = files.upload()
template_filename = list(uploaded.keys())[0]
t = cv2.imdecode(np.frombuffer(uploaded[template_filename], np.uint8), cv2.IMREAD_UNCHANGED)
tg = cv2.cvtColor(t, cv2.COLOR_BGR2GRAY)
cv2_imshow(t)

# Taking the height and width of the template image
h = tg.shape[0]
w = tg.shape[1]


In [ ]:
# Loading the main image and gray-scaling it

print("Upload the main image:")
uploaded = files.upload()
main_filename = list(uploaded.keys())[0]
m = cv2.imdecode(np.frombuffer(uploaded[main_filename], np.uint8), cv2.IMREAD_UNCHANGED)
mg = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
cv2_imshow(m)

In [ ]:
# A nested for loop the examine all possible template cases to find any possible match

# The outer loop interate on the specifyed scale range
# Narrowing the scale range lowers the run time but also accuracy
for scale in frange(1, 100, 0.5):
    print("Scale = ", scale)

    # Resize the template to the specifyed scale
    resized_template = cv2.resize(tg, (int(w * scale), int(h * scale)))

    # Checking if the new size of template is less than size of the main image
    if resized_template.shape > mg.shape:
        break

    # The inner loop interate on the specifyed angle range
    # Narrowing the angle range lowers the run time but also accuracy
    for angle in frange(0, 360, 10):
        print("Angle = ", angle)

        # Rotate the resized template to the specifyed angle
        rotated_template = ndimage.rotate(resized_template, angle)

        # Checking if the new rotation of template made the width or height exceed the width or height of the main image
        if rotated_template.shape[0] > mg.shape[0] or rotated_template.shape[1] > mg.shape[1]:
            break

        # Ready made function to compare the template to main image
        result = cv2.matchTemplate(mg, rotated_template, cv2.TM_CCOEFF_NORMED)

        # Ready made function to extract the main values resulted from the template matching
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        print(">", max_val)

        # If the score is above the threshold, a match is found
        if max_val > best_val:
            best_val = max_val
            best_match = max_loc
            best_scale = scale
            best_angle = angle
            print("==>>", best_val, best_scale, best_angle)

    print("---------------------------------")

In [ ]:
print(best_match, best_val, best_scale, best_angle)

# Edit the template to have the best scale an angle
rt = cv2.resize(tg, (int(w * best_scale), int(h * best_scale)))
rotatedt = ndimage.rotate(rt, best_angle)

In [ ]:
# Calculating the new width and height of the template

scaled_h = int(h * best_scale)
scaled_w = int(w * best_scale)

# print results
print("Old width and height:", w, h)
print("Rescale width and height:", scaled_w, scaled_h)

In [ ]:
# Seaching for multiple instances of the template in the main image
res = cv2.matchTemplate(mg, rotatedt, cv2.TM_CCOEFF_NORMED)

# Store the coordinates of matched areas in a array
loc = np.where(res >= 0.99)

# Draw a rectangle around every matched region.
for pt in zip(*loc[::-1]):
    print(pt)
    cv2.rectangle(m, pt, (int((pt[0] + scaled_w)), int((pt[1] + scaled_h))), (0, 0, 0), 2)

# Show the final image with the matched areas.
cv2_imshow(m)